In [1]:
#!/usr/bin/env python3

In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

2024-04-08 14:56:41.970887: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-08 14:56:41.971125: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-08 14:56:41.973192: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-08 14:56:41.999178: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-08 14:56:42.447764: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warnin

In [3]:
IMAGE_SIZE = (224, 224)
BATCH_SIZE = 32
NUM_CLASSES = 3
EPOCHS = 50

In [4]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2  # Splitting training data for validation
)

In [5]:
train_generator = train_datagen.flow_from_directory(
    'Dataset',
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training'  # Use training data
)

Found 151 images belonging to 3 classes.


In [6]:
validation_generator = train_datagen.flow_from_directory(
    'Dataset',
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation'  # Use validation data
)

Found 36 images belonging to 3 classes.


In [7]:
from tensorflow.keras.applications.inception_v3 import InceptionV3


base_model = InceptionV3(weights='imagenet', include_top=False, pooling='avg')


2024-04-08 14:56:42.862186: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-04-08 14:56:42.862727: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2251] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [8]:
for layer in base_model.layers:
    layer.trainable = False

In [9]:
x = Flatten()(base_model.output)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.2)(x)

# Add a final sigmoid layer with 1 node for classification output
x = Dense(NUM_CLASSES, activation='sigmoid')(x)

In [10]:
model = Model(inputs=base_model.input, outputs=x)
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])


In [11]:
from scipy import ndimage 

history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // BATCH_SIZE,
    epochs=EPOCHS,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // BATCH_SIZE
)

Epoch 1/50


/home/jatin/.local/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:120: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


4/4 ━━━━━━━━━━━━━━━━━━━━ 7s 904ms/step - accuracy: 0.4938 - loss: 3.8528 - val_accuracy: 0.7188 - val_loss: 1.4316
Epoch 2/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 0.6957 - loss: 0.9085 - val_accuracy: 0.2500 - val_loss: 2.3184
Epoch 3/50


2024-04-08 14:56:51.723149: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
/usr/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 319ms/step - accuracy: 0.6814 - loss: 1.1601 - val_accuracy: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 4/50


...

2024-04-08 14:58:06.756843: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 215ms/step - accuracy: 0.8438 - loss: 0.2216 - val_accuracy: 0.7188 - val_loss: 0.8105
Epoch 47/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 3s 361ms/step - accuracy: 0.9527 - loss: 0.1627 - val_accuracy: 0.7500 - val_loss: 0.2576
Epoch 48/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.9375 - loss: 0.1064 - val_accuracy: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 49/50


2024-04-08 14:58:10.374643: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
2024-04-08 14:58:10.380831: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


4/4 ━━━━━━━━━━━━━━━━━━━━ 4s 629ms/step - accuracy: 0.9203 - loss: 0.1970 - val_accuracy: 0.8125 - val_loss: 0.5321
Epoch 50/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - accuracy: 0.9688 - loss: 0.0909 - val_accuracy: 0.7500 - val_loss: 0.6437


2024-04-08 14:58:14.317935: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


In [12]:
# model.save('rice_classification_model.h5')

In [13]:
validation_loss, validation_accuracy = model.evaluate(validation_generator)

print("Validation Loss:", validation_loss)
print("Validation Accuracy:", validation_accuracy)

2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 67ms/step - accuracy: 0.8079 - loss: 0.6072
Validation Loss: 0.5723828077316284
Validation Accuracy: 0.8055555820465088
